In [1]:
# Se importan las librerias correspondientes
import nltk
from nltk.corpus import stopwords
import pandas
import numpy as np
import utils_pln as utl
from sklearn.cross_validation import train_test_split

### 1. Importamos el corpus de comentarios

Pandas es una biblioteca de código abierto implementada en Python la cual permite realizar una fácil manipulación y análisis de los datos.
Ésta se utilizó para cargar los datos en memoria y realizar un breve análisis de los mismos.

In [2]:
# Se carga en 'datos' el archivo csv en memoria
datos = pandas.read_csv("comentarios_peliculas.csv", skiprows=1, delimiter=';', skip_blank_lines=True, encoding='utf-8')

In [3]:
elementos_subjetivos = utl.diccionarioElementosSubjetivos("elementos_subjetivos.txt")

### 2. Características del corpus
Para conocer acerca del corpus sobre el cual se trabajará se realiza un breve análisis de los datos. Para ello se obtiene la cantidad de filas y columnas que el dataset posee; donde las filas corresponden a los comentarios y las columnas a los atributos asociados a los mismos. Por otro lado se obtiene la cantidad de peliculas del corpus, la cantidad de comentarios para cada película, en el que se realiza un ploteo, entre otros. A su vez se listan esos atributos junto a su tipo asociado. Una vez hecho esto se procede a chequear de que los atributos de los comentarios tengan o no valores faltantes, ya que en caso de tenerlos hay que tomar una decisión en base a que hacer con los mismos (existen varias técnicas para este tratamiento). 

In [4]:
# Se obtiene cantidad peliculas, de comentarios y cantidad de atributos
cantPeliculas = len(datos.ix[:,0].unique())
cantComentarios = len(datos.index)
cantAtributos = len(datos.columns)

# Se imprimen los datos obtenidos anteriormente junto a sus tipos de datos
print ("Hay una cantidad de " + str(cantPeliculas) + " películas en el corpus.")
print ("El corpus posee " + str(cantComentarios) + " comentarios con " + str(cantAtributos) + " atributos por cada uno de ellos.")

# Se imprime la cantidad de comentarios para cada pelicula
print ("\nCantidad de comentarios asociados a cada película:\n")
print (datos.ix[:,0].value_counts())

Hay una cantidad de 127 películas en el corpus.
El corpus posee 1447 comentarios con 9 atributos por cada uno de ellos.

Cantidad de comentarios asociados a cada película:

Relatos salvajes                                      69
A 60 km/h                                             55
Mr. Kaplan                                            49
Cincuenta sombras de Grey                             46
Sin hijos                                             41
El Gran Hotel Budapest                                36
23 segundos                                           35
Intensa-Mente                                         32
Directo al corazón                                    28
Tus padres volverán                                   26
El planeta de los simios: confrontación               26
Betibú                                                24
Noé                                                   21
Maléfica                                              21
Guardianes de la Galaxia     

In [5]:
# Se importa la libreria para utilizar plot
# Además se plotea en el própio notebook y no como una ventana nueva
%matplotlib inline
from pylab import *

# Se realiza una gráfica de cantidad de comentarios en función de su clasificación
gruposCalificacion = datos.groupby([u'Calificación']).groups
keyClasificacion = gruposCalificacion.keys()
cantClasificacion = []
for k in keyClasificacion:
    cantClasificacion.append(len(gruposCalificacion.get(k)))    
pos = arange(len(keyClasificacion)) + 0.5 

figure(1)
barh(pos,cantClasificacion, align='center')
yticks(pos, keyClasificacion)
xlabel('Cantidad de comentarios')
ylabel(u'Clasificación')
title(u'Grafica cantidad de comentarios por clasificación')
grid(True)
show()

TypeError: 'dict_keys' object does not support indexing

TypeError: 'dict_keys' object does not support indexing

In [6]:
#Se imprime la cantidad de comentarios asociados a cada clase (es decir cantidad de comentarios por calificación)
print ("\nCantidad de comentarios asociados a cada calificación:\n")
print (datos.ix[:,7].value_counts())


Cantidad de comentarios asociados a cada calificación:

5    505
4    354
1    225
3    222
2    141
dtype: int64


In [7]:
print ("A continuación se listan los atributos con su tipo asociado:\n")
print (datos.dtypes)

A continuación se listan los atributos con su tipo asociado:

Título          object
Año              int64
Estrellas        int64
Género          object
Duración        object
Reseña          object
ComTexto        object
Calificación     int64
ComFechaHora    object
dtype: object


In [8]:
# Se chequea si los atributos poseen o no valores faltantes
# Para ello se recorren todos los atributos (columnas de 'datos')
for atributo in datos.columns:
    
    # Se obtiene la cantidad de valores distinto de vacio del atributo 
    cantValoresAtributo = datos[atributo].describe()['count']
    
    # Si hay menos que la cantidad de comentarios de 'datos' -> hay valores faltantes
    if(cantComentarios > cantValoresAtributo):
        print ("Hay valores faltantes para el atributo " + atributo)
    else:
        print ("No hay valores faltantes para el tributo " + atributo)

No hay valores faltantes para el tributo Título
No hay valores faltantes para el tributo Año
No hay valores faltantes para el tributo Estrellas
No hay valores faltantes para el tributo Género
No hay valores faltantes para el tributo Duración
No hay valores faltantes para el tributo Reseña
No hay valores faltantes para el tributo ComTexto
No hay valores faltantes para el tributo Calificación
No hay valores faltantes para el tributo ComFechaHora


### 3. Descartamos columnas innecesarias
En esta sección se descartan las columnas (atributos) que se consideran innecesarios. En particular las columnas que creemos se consideran necesarias son solamente las columnas de los comentarios y los puntajes asociados a dichos comentarios. Por lo que se procede a eliminar el resto.

In [9]:
# Se descartan de 'datos' las columnas innecesarias, dejando solamente los comentarios y sus calificaciones
datos.drop(datos.columns[[0,1,2,3,4,5,8]],inplace=True,axis=1)
comentarios_peliculas = datos
# Se imprime un resumen de los datos
print (comentarios_peliculas)

                                               ComTexto  Calificación
0     Muchas gracias, vi online los primeros 4, lueg...             4
1           La mejor serie desde Los Soprano y The Wire             5
2     Llega un punto en el que aburre. Es un plato b...             2
3     En realidad no es una respuesta , es una corre...             1
4     Ahora entiendo menos. El libro en que se basa ...             1
5     Una pena desperdiciar un buen elenco y obviame...             1
6     MAGISTRAL!! excelente la recreación de época. ...             5
7                             es una película mui padre             5
8     EXCELENTE! GENIAL! Esperaba ir a ver con mi hi...             5
9     Muy, muy buena...para grandes y chicos. Tiene ...             5
10    Excelente película, muy original y divertida. ...             5
11    Muy buena película, muy graciosa y de humor ba...             4
12    Muy linda película. La fui a ver con mi hija, ...             4
13    Linda película

### 3. Preprocesamiento de los comentarios
El corpus sobre el cual se trabaja puede tener contener datos con ruido en su contenido. Es decir, diversos elementos que están mezclados en el contenido pero que además de no aportar nada a la tarea de clasificación, generan ruido dificultando las tareas posteriores. Para ello entonces se procede a realizar el preprocesamiento de los comentarios.

In [11]:
comentarios_peliculas = utl.convert_to_list(utl.depurar_comentarios(comentarios_peliculas))

AttributeError: 'list' object has no attribute 'ix'

### 4. Separamos entrenamiento y testeo 
Una vez de haber llegado a este punto tenemos los datos tokenizados en la variable listaTuplas. Habrá entonces que dividir dicha lista en conjuntos de entrenamiento y testeo, para primero entrenar el algoritmo y luego testearlo.
Para poder entrenar y testear un algoritmo de aprendizaje, como se mencionó, es necesario primero particionar los datos en dos conjuntos disjuntos de entrenamiento y testeo. Se separarán aleatoriamente un 20% de los datos para testeo y el 80% restante se utilizará para el entrenamiento. Para esto se utilizó la función train_test_split del paquete cross_validation de sklearn.

In [13]:
filtroSW = []
filtroAC = []

dominio_cine_peliculas = open('terminosNoValorativosAmbitoCine.txt').read()
palabras_f = utl.palabras_mas_frecuentes(1500,comentarios_peliculas)
nltk_stopwords = stopwords.words('spanish')

#Aplico distintos filtros a las palabras de los comentarios
for i in range(0,len(comentarios_peliculas)):
    #elimino stopwords
    palabras_frecuentes = [w for w in comentarios_peliculas[i][0].split(" ") if not w in palabras_f]
    filtroSW = [w for w in palabras_frecuentes if not w in nltk_stopwords]
    filtroAC = [w for w in filtroSW if not w in dominio_cine_peliculas]
    comentarios_peliculas[i] = (" ".join(filtroAC),comentarios_peliculas[i][1])

In [14]:
# Se divide el conjunto de datos en train (80%) y test (20%)
datos_train, datos_test = train_test_split(comentarios_peliculas, test_size=0.2)

# Se chequean las cantidades de la división
print (len(datos_train))
print (len(datos_test))

1157
290


### 4. Tokenización
En esta sección se realiza el proceso de tokenización de los textos utilizando la biblioteca NLTK. Además se convierte a un formato suceptible para la clasificación. Es decir, se toquenizan los comentarios y se calcula el diccionario de palabras frecuentes del mismo. En base a ello y al valor de clasificación asociado se crea una tupla la cual corresponde al comentario analizado. Luego estas tuplas (una por cada comentario) son la entrada para el entrenamiento del algoritmo Entropía Máxima provisto por NLTK.

In [15]:
# Se imprime el primer comentario junto a su clasificacion
print (datos_train[0][0])

datos_train = utl.tokenizar_nltk(datos_train)

# Si todo esta bien al imprimir el primer elemento de la lista
# Se deberia ver el diccionario mas la clasificacion asociada al comentario 
print (datos_train[0])

Muy buena película; volvió Al Pacino pensé nunca iba volver ver. Super recomendable.
({';': 1, 'recomendable': 1, 'volver': 1, '.': 2, 'nunca': 1, 'super': 1, 'pacino': 1, 'buena': 1, 'ver': 1, 'muy': 1, 'volvió': 1, 'pensé': 1, 'película': 1, 'iba': 1, 'al': 1}, 'pos')


### 6. Se entrena el algoritmo de máxima entropia y se lo evalua
Una vez de tener los datos en un formato aceptable para los algoritmos de entrenamiento y particionado los mismos en test y train, se procede a realizar el entrenamiento del algoritmo de Máxima Entropía. Una vez realizado dicho entrenamiento se procede a evaluarlo.

In [16]:
# En la siguiente linea se entrena el algoritmo con 7 iteraciones
# Obs. El entrenamiento es bastante lento, habría que llegar a un valor de iteraciones adecuado
clf_max_ent = nltk.classify.MaxentClassifier.train(datos_train,max_iter=8)

  ==> Training (8 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.596
             2          -0.70759        0.765
             3          -0.55691        0.881
             4          -0.46771        0.920
             5          -0.40759        0.941
             6          -0.36367        0.954
             7          -0.32984        0.965
         Final          -0.30280        0.968


In [17]:
# Funcion que dada una distribución de probabilidad de las clasificaciones
# devuelve la clasificacion que tiene mayor probabilidad
def getClasificacion(pdist):
    
    # Se inicializa la clasificacion
    clasificacion = 0;
    prob = 0
    
    # Por cada clasificacion posible, la comparo con la inicializacion
    # Me quedo con la mas grande
    for i in range(1,6):
        if( pdist.prob(i) > prob):
            clasificacion = i
            prob = pdist.prob(i)

    return clasificacion

In [18]:
def getTasa(clf,datos_test):
    # Se define la variable de aciertos
    aciertos = 0
    
    salidaClasificador = []
    salida = []

    # Para cada comentario del conjunto de testeo se evalua segun el algoritmo entrenado
    for comentario in datos_test:

        # Se obtiene la clasificacion del algoritmo para el comentario
        clasificacion = clf.classify(comentario[0])
        salidaClasificador.append(clasificacion)
        salida.append(comentario[1])
        
        # En caso que la clasificacion sea la correcta se aumenta el acierto
        if(clasificacion == comentario[1]):
            aciertos += 1
    
    cm = nltk.ConfusionMatrix(salidaClasificador, salida)
    print(cm)

    return float(aciertos)/len(datos_test)

In [19]:
print ("Tase de Max Ent: " + str(getTasa(clf_max_ent,utl.tokenizar_nltk(datos_test))))

    |   n   n   p |
    |   e   e   o |
    |   g   u   s |
----+-------------+
neg | <39>  7   5 |
neu |   3  <4>  5 |
pos |  30  38<159>|
----+-------------+
(row = reference; col = test)

Tase de Max Ent: 0.696551724137931


In [20]:
clf_max_ent.show_most_informative_features(10)

  -4.008 excelente==1 and label is 'neu'
   3.907 buenasa==1 and label is 'pos'
   3.907 siniestra==1 and label is 'neg'
   3.907 divinaa==1 and label is 'pos'
   3.907 melor==1 and label is 'pos'
   3.907 cool==1 and label is 'pos'
   3.907 sarpada==1 and label is 'pos'
  -3.199 excelente==1 and label is 'neg'
   2.809 good==1 and label is 'pos'
   2.793 puntaje==1 and label is 'pos'


In [27]:
def codificarClasificacionesSubjetivos():
    lista = []
    for t in elementos_subjetivos:

        clasificacion = 'neg'
        if elementos_subjetivos[t] == 3:
            clasificacion = 'pos'
            
        lista.append(({t:1},clasificacion))
    
    return lista

def getPositivos():
    lista = []
    for t in elementos_subjetivos:

        #clasificacion = 'neg'
        if elementos_subjetivos[t] == 3:
            lista.append(t)
    
    return lista

def getNegativos():
    lista = []
    for t in elementos_subjetivos:

        #clasificacion = 'neg'
        if elementos_subjetivos[t] != 3:
            lista.append(t)
    
    return lista

In [28]:
#print(codificarClasificacionesSubjetivos())
#clf_max_ent.train(codificarClasificacionesSubjetivos(), max_iter = 7)

In [29]:
subPositivos = getPositivos()
subNegativos = getNegativos()


In [30]:
def atributos(diccionario,SUBP,SUBN):
    atributos = {}
    cantPositivos = 0
    cantNegativos = 0
    cantPalabrasNeutras = 0
    comentario = list(diccionario.keys())[0]
    largoComentario = len(comentario)
    #GRAL
    
    if (largoComentario > 14): 
        atributos["comentario_largo"]=True
        atributos["comentario_corto"]=False
    else: 
        atributos["comentario_corto"]=True
        atributos["comentario_largo"]=False

    for w in SUBP:
        if w in comentario: atributos[w]=True
        else: atributos[w]=False
        cantPositivos += comentario.count(w)
    for w in SUBN:
        if w in comentario: atributos[w]=True
        else: atributos[w]=False
        cantNegativos += comentario.count(w)

    #POSITIVOS
    if (cantPositivos < 1): atributos["hay_Positivos"] = False
    else: atributos["hay_Positivos"] = True

    if (cantPositivos > 3): atributos["muchos_Positivos"] = True
    else: atributos["muchos_Positivos"] = False

    #NEGATIVOS
    if (cantNegativos < 1): atributos["hay_Negativos"] = False
    else: atributos["hay_Negativos"] = True

    if (cantNegativos > 3): atributos["muchos_Negativos"] = True
    else: atributos["muchos_Negativos"] = False

    #NEUTRAS
    for w in comentario:
        if ((w not in SUBN) & (w not in SUBP)): 
            #atributos[w] = 1
            cantPalabrasNeutras +=1

    if (cantPalabrasNeutras <= 5): atributos["pocasPalabrasNeutras"] = True
    else: atributos["pocasPalabrasNeutras"] = False

    return atributos


In [31]:
training_set = [(atributos(c, subPositivos, subNegativos), p) for (c,p) in datos_train]

AttributeError: 'str' object has no attribute 'keys'

In [302]:
test_set = [(atributos(c, subPositivos, subNegativos), p) for (c,p) in datos_test]

IndexError: list index out of range

In [ ]:
clf_max_ent = nltk.classify.MaxentClassifier.train(training_set,max_iter=7)
print(nltk.classify.accuracy(clf_max_ent, test_set))
clf_max_ent.show_most_informative_features()

In [ ]:
salidaClasificador = []
salida = []
for c in test_set:
    salidaClasificador.append(clf_max_ent.classify(c[0]))
    salida.append(c[1])

cm = nltk.ConfusionMatrix(salidaClasificador, salida)
print(cm)
